# Blackjack with AI player

In [63]:
import random

In [64]:
SUITS = ['club', 'spade', 'heart', 'diamond']
RANKS = ['2','3','4','5','6','7','8','9','10','J','Q','K','A']

# Card class keeps:
    # suit - The suit of the card [club, spade, heart, diamond]
    # rank - The rank of the card [2,3,4,5,6,7,8,9,10,J,Q,K,A]
    # value - The point value associated with the rank
    # hidden - faceDown = True ; faceUp = false
    # isAce - True if the card is an Ace
class Card(object):
    
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        
        if rank == 'A':
            self.value = 11
            self.isAce = True
        elif rank in ['J','Q','K']:
            self.value = 10
            self.isAce = False
        else:
            self.value = int(rank) # 2-10
            self.isAce = False
            
        self.hidden = False # Dealer will have one card hidden from the player
         
    def __str__(self):
        if self.hidden:
            return str(tuple('X'))
        else: 
            return str((self.suit, self.rank))
    
    def hideCard(self):
        self.hidden = True
        
    def showCard(self):
        self.hidden = False

# Deck class keeps:
    # cards - list of cards remaining in the deck
class Deck(object):
    
    def __init__(self, nDecks = 1):
        self.buildDeck(nDecks)
        self.prepareDeck()
    
    def buildDeck(self, nDecks):
        self.cards = [Card(suit,rank) for i in range(nDecks) for suit in SUITS for rank in RANKS]
    
    def prepareDeck(self):
        for i in range(7):
            self.shuffle()
            
    def shuffle(self):
        random.shuffle(self.cards)
        
    def draw(self):
        card = self.cards.pop(0)
        return card
    
    def __len__(self):
        return len(self.cards)
    
    def __str__(self):
        return str([str(card) for card in self.cards])
    
# Hand class keeps
    # hand - cards in the hand
    # bet - the bet placed on the hand
class Hand(object):
    
    def __init__(self, card = None, bet = 50):
        self.aces = 0
        if card:
            self.hand = [card]
        else:
            self.hand = []
        self.bet = bet
        
    def addCard(self, card):
        self.hand.append(card)     
            
    def removeCard(self):
        return self.hand.pop()
    
    @property
    def value(self):
        value = sum(card.value for card in self.hand)
        aces = sum(card.isAce for card in self.hand)  
        # Aces are initially given a value of 11, they are ammended to 1 if the total valaue is > 21
        while (value > 21) and aces: 
            value -= 10
            aces -= 1
        return value
        
    def __str__(self):
        return str([str(card) for card in self.hand])
        

In [65]:
# Player class keeps:
    # money
    # hands
class Player:
    
    def __init__(self, money = 5000):
        self.money = money
        self.hands = [Hand()]
        
    def checkBust(self):
        if self.hand.value > 21:
            return True
        return False
    
    def showHands(self):
        print("Player has %d hands: \n" % len(self.hands))
        for hand in self.hands:
            print(hand)
        print()
    
    def stats(self):
        print("Player has $%d left." % self.money)
        # possible other stats later on
        

# Dealer class keeps:
    # deck
    # hand
    # isBust
class Dealer:
    
    def __init__(self, deck, Player):
        self.deck = deck
        self.hand = Hand()
        self.Player = Player
    
    def deal(self):
        for i in range(2): # deal each player, including the dealer, 2 cards
            self.hand.addCard(self.deck.draw()) # Deal the dealer a card
            if i == 0: # dealers first card is face down
                self.hand.hand[0].hideCard() 
            self.Player.hands[0].addCard(self.deck.draw()) # Deal the player a card        
        
    def hit(self):
        self.hand.addCard(self.deck.draw())
            
    def stand(self):
        return self.hand
    
    def checkBust(self):
        if self.hand.value > 21:
            return True
        return False
    
    def showHands(self):
        print('Dealers Hand:')
        print(hand)
    
    
def validMoves():
    return 0

def makeMoves():
    return 0

In [66]:
class Blackjack:
    
    def __init__(self, Player):
        self.Player = Player
        self.Deck = Deck(6)
        self.Dealer = Dealer(self.Deck, Player)
        self.Dealer.deal()
        
    def newHand(self):
        self.Player.hands = [Hand()]
        self.Dealer.hand = Hand()
        self.Dealer.deal()
        
    def newDeck(self):
        if len(self.Deck) <= 52:
            self.Deck = Deck(6)
            
    def gameStatus(self):
        # player bust
        if self.Player.checkBust():
            return -1
        # dealer bust
        if self.Dealer.checkBust():
            return 1
        # draw
        if self.Player.hand.value == self.Dealer.hand.value:
            return 0
        # player win
        if self.Player.hand.value > self.Dealer.hand.value:
            return 1
        # dealer win
        if self.Player.hand.value < self.Dealer.hand.value:
            return -1
        

## Testing

In [67]:
deck = Deck(1)
print(len(deck))
print(str(deck))
print(str(deck.draw()))
print(str(deck.draw()))
print(len(deck))

52
["('diamond', '8')", "('club', '9')", "('spade', '4')", "('spade', '6')", "('spade', '3')", "('club', '5')", "('heart', '9')", "('diamond', '5')", "('diamond', 'J')", "('heart', '8')", "('club', '3')", "('spade', '8')", "('club', '10')", "('club', 'J')", "('club', '6')", "('diamond', '9')", "('spade', '10')", "('diamond', 'K')", "('club', '8')", "('spade', '9')", "('heart', '3')", "('heart', '4')", "('diamond', '7')", "('heart', '6')", "('spade', 'J')", "('heart', 'Q')", "('heart', 'K')", "('club', '7')", "('spade', '7')", "('diamond', '6')", "('diamond', 'A')", "('heart', '2')", "('heart', '7')", "('diamond', '2')", "('spade', 'K')", "('diamond', '3')", "('club', 'Q')", "('spade', '2')", "('heart', '10')", "('spade', 'A')", "('heart', '5')", "('club', '4')", "('heart', 'A')", "('diamond', '10')", "('spade', 'Q')", "('diamond', '4')", "('heart', 'J')", "('club', 'A')", "('spade', '5')", "('club', '2')", "('club', 'K')", "('diamond', 'Q')"]
('diamond', '8')
('club', '9')
50


In [68]:
deck = Deck(3)
print(len(deck))
hand = Hand()
hand.addCard(deck.draw())
print(str(hand))
hand.addCard(deck.draw())
print(str(hand))
hand.addCard(deck.draw())
print(str(hand))
print(str(hand.value))

156
["('spade', '4')"]
["('spade', '4')", "('spade', '5')"]
["('spade', '4')", "('spade', '5')", "('spade', '7')"]
16


In [71]:
player = Player()
game = Blackjack(player)
print(len(game.Deck))
game.Player.showHands()
game.Dealer.showHands()

308
Player has 1 hands: 

["('club', '8')", "('club', '3')"]

Dealers Hand:
["('spade', '4')", "('spade', '5')", "('spade', '7')"]
